# Convolutional Neural Networks
You should build an end-to-end machine learning pipeline using a convolutional neural network model. In particular, you should do the following:
- Load the `mnist` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the datasets folder.
- Split the dataset into training and test sets using [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
- Build an end-to-end machine learning pipeline, including a [convolutional neural network](https://keras.io/examples/vision/mnist_convnet/) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

1. change mnist data with our own mnist data
2. Do experimentation with keras Sequential function: changing optimization, convolutional, pooling, better validation score
3. final experimentation to be reported

Reading the Dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/m-mahdavi/teaching/refs/heads/main/datasets/mnist.csv')

Splitting the Dataset

In [3]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((3200, 786), (800, 786))

EDA

In [4]:
df_train.head()

,id,class,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
3994,13260,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
423,10953,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2991,37374,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,31597,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506,69405,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train.isnull().sum()

,0
id,0
class,0
pixel1,0
pixel2,0
pixel3,0
...,...
pixel780,0
pixel781,0
pixel782,0
pixel783,0


Data Preprocessing

In [6]:
x_train = df_train.drop(['id','class'], axis=1).values
y_train = df_train['class'].values

x_test = df_test.drop(['id','class'], axis=1).values
y_test = df_test['class'].values

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3200, 784), (3200,), (800, 784), (800,))

Feature Engineering

In [8]:
sk = StandardScaler()
x_train = sk.fit_transform(x_train)
x_test = sk.transform(x_test)

Reshaping

In [9]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

Model Training

In [13]:
cnn = keras.Sequential()
cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
Dropout(0.4),
BatchNormalization(),
cnn.add(Dense(64, activation='relu'))
Dropout(0.4),
BatchNormalization(),
cnn.add(Dense(10, activation='softmax'))

In [14]:
cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        51,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,178 (254.60 KB)

 Trainable params: 65,178 (254.60 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.fit(x_train, y_train, batch_size=128, epochs=25, validation_split=0.1, callbacks = EarlyStopping(patience=5, monitor='val_loss'))

Epoch 1/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.1290 - loss: 2.3012 - val_accuracy: 0.1969 - val_loss: 2.2864
Epoch 2/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.1271 - loss: 2.2823 - val_accuracy: 0.3781 - val_loss: 2.1988
Epoch 3/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.3282 - loss: 2.1312 - val_accuracy: 0.4625 - val_loss: 1.7327
Epoch 4/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.5075 - loss: 1.6040 - val_accuracy: 0.6375 - val_loss: 1.2013
Epoch 5/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.6435 - loss: 1.1062 - val_accuracy: 0.7437 - val_loss: 0.8679
Epoch 6/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.7558 - loss: 0.8329 - val_accuracy: 0.8125 - val_loss: 0.7188
Epoch 7/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.7795 - loss: 0.7275 - val_accuracy: 0.8000 - val_loss: 0.6474
Epoch 8/25
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 112ms/step - accuracy: 0.8270 - loss: 0.5842 - val_accuracy: 0.8406 - 